In [ ]:
# Imports used in this notebook.
import ee
import os
import sys
import time
import shutil
import pandas as pd
import seaborn as sea
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from ee import batch
from matplotlib import cm
from google.colab import drive

# Must authenticate your EE account before use of the package.
#ee.Authenticate()
#ee.Initialize()
ee.Authenticate()
ee.Initialize()
# This is how we can access our drive to get the correlation product.
drive.mount('/content/gdrive')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=wlyeIQ1MfQwNhNsRamzWlTlf3UFWj3feJhYNAVpeNlk&tc=RCipn2ZSYHPKcvZ890pIDTf5-kyj7b7SLo7wYpvtwtY&cc=AJa9EK9xauYIvoBJNd6T7wQ5sOcd7sD0YeSUHlTPI5c

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk5W7W3Jo_iTJMlehaVfGSx2Atx884vPOd27VK4mN9nY6hXyL_Edurw

Successfully saved authorization token.
Mounted at /content/gdrive


# Set the AOI.

In [ ]:
#Florida DEP bounds
#Indian River Lagoon
siteName = 'Indian River Lagoon'
siteName_2char = 'IRL'
upperLeft = [-80.763739, 28.621909]
lowerRight = [-80.503601,  27.605387]
# upperLeft = [-80.763739, 28.621909]
# lowerRight = [-80.533771,   28.063011]

# Define a region of interest to filter our collection.
roi = ee.Geometry.Rectangle(upperLeft[0], upperLeft[1], lowerRight[0], lowerRight[1])

# Create a list of dates to use for image extraction

In [ ]:
# List of dates.
# List of dates.
date_list = [
ee.Date('2020-01-01'),
ee.Date('2020-01-06'),
ee.Date('2020-01-16'),
ee.Date('2020-02-03'),
ee.Date('2020-02-08'),
ee.Date('2020-03-01'),
ee.Date('2020-03-14'),
ee.Date('2020-03-24'),
ee.Date('2020-03-29'),
ee.Date('2020-04-03'),
ee.Date('2020-04-08'),
ee.Date('2020-05-05'),
ee.Date('2020-05-20'),
ee.Date('2020-06-24'),
ee.Date('2020-06-29'),
ee.Date('2020-07-14'),
ee.Date('2020-07-19'),
ee.Date('2020-07-24'),
ee.Date('2020-08-08'),
ee.Date('2020-08-13'),
ee.Date('2020-09-02'),
ee.Date('2020-09-17'),
ee.Date('2020-10-15'),
ee.Date('2020-10-27'),
ee.Date('2020-11-24'),
ee.Date('2020-12-09'),
ee.Date('2020-12-21'),
ee.Date('2020-12-29'),
ee.Date('2021-01-05'),
ee.Date('2021-01-20'),
ee.Date('2021-01-28'),
ee.Date('2021-02-02'),
ee.Date('2021-02-19'),
ee.Date('2021-02-22'),
ee.Date('2021-02-24'),
ee.Date('2021-02-27'),
ee.Date('2021-03-01'),
ee.Date('2021-03-04'),
ee.Date('2021-03-11'),
ee.Date('2021-03-14'),
ee.Date('2021-03-19'),
ee.Date('2021-03-24'),
ee.Date('2021-04-05'),
ee.Date('2021-04-10'),
ee.Date('2021-04-30'),
ee.Date('2021-05-05'),
ee.Date('2021-05-10'),
ee.Date('2021-05-15'),
ee.Date('2021-05-25'),
ee.Date('2021-06-09'),
ee.Date('2021-07-19'),
ee.Date('2021-07-29'),
ee.Date('2021-09-07'),
ee.Date('2021-09-22'),
ee.Date('2021-09-27'),
ee.Date('2021-09-30'),
ee.Date('2021-10-02'),
ee.Date('2021-10-12'),
ee.Date('2021-11-16'),
ee.Date('2021-11-26'),
ee.Date('2021-11-29'),
ee.Date('2021-12-04'),
ee.Date('2021-12-06'),
ee.Date('2021-12-09'),
ee.Date('2021-12-16'),
ee.Date('2021-12-24'),
ee.Date('2021-12-26'),
ee.Date('2021-12-29'),
ee.Date('2022-01-03'),
ee.Date('2022-01-08'),
ee.Date('2022-01-15'),
ee.Date('2022-01-18'),
ee.Date('2022-01-20'),
ee.Date('2022-01-30'),
ee.Date('2022-02-04'),
ee.Date('2022-02-12'),
ee.Date('2022-02-14'),
ee.Date('2022-02-22'),
ee.Date('2022-02-24'),
ee.Date('2022-03-04'),
ee.Date('2022-03-21'),
ee.Date('2022-03-26'),
ee.Date('2022-04-05'),
ee.Date('2022-04-10'),
ee.Date('2022-04-25'),
ee.Date('2022-05-05'),
ee.Date('2022-05-15'),
ee.Date('2022-06-24'),
ee.Date('2022-07-19'),
ee.Date('2022-07-29'),
ee.Date('2022-08-18'),
ee.Date('2022-09-07'),
ee.Date('2022-09-12'),
ee.Date('2022-09-22'),
ee.Date('2022-10-02'),
ee.Date('2022-10-05'),
ee.Date('2022-10-07'),
ee.Date('2022-10-25'),
ee.Date('2022-10-27'),
ee.Date('2022-10-30'),
ee.Date('2022-11-01'),
ee.Date('2022-11-04'),
ee.Date('2022-11-11'),
ee.Date('2022-11-29'),
ee.Date('2022-12-01'),
ee.Date('2022-12-04'),
ee.Date('2022-12-06'),
ee.Date('2022-12-09'),
ee.Date('2022-12-29'),
]


# Select our Sentinel-2 imagery


In [ ]:
im_list = []

# Start Standard code for processing images from a collection.
for date in date_list:

  # Bring in our imagery.
  coll = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
              .filterBounds(roi)
              .filterDate(date, date.advance(1,'day')))

  # Clip our input image to our region of interest.
  water = coll.mosaic().clip(roi).set({'name':coll.first().get('system:index')})

  im_list += [water]

In [ ]:
im_list

# Begin analysis

In [ ]:
# output_folder = '/content/gdrive/Shared drives/Asgard/FL_DEP_Seagrass_KSU_ST/Sentinel2/part2'

output_folder = '/content/gdrive/My Drive/test_FL_DEP'

# Define the L8 bands to extract
non_ee_band_list = ['B1','B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A']
# For reductions.
native_res = 10
maxP = 1e10

# Array of wavelength corrections for Landsat 8 sensor.
wl = ee.Array([443.9,496.6,560.0,664.5,703.9,740.2,782.5,835.1,864.8])
# wl = ee.Array([442.7,492.7,559.8,664.4,704.1,740.5,782.8,832.8,864.7])
# Name the band combo for outputs.
bandCombo="B2-B8A"

""" Bands and Wavelength Array to select. You can add or subtract any you'd like here and the code
    below will account for the change throughout the workflow. """

# Use this for the derivative calculation.
band_list_early_length = len(non_ee_band_list)
# We use this everywhere else after the derivative calc (original band list x2).
#band_list_length = band_list_early_length *2
band_list_length = band_list_early_length

# Use this in a couple for loops throughout the code.
band_list_forLoops = range(band_list_length)

In [ ]:
def derivative_images(water):

  """This section is devoted to modifying the image, making it suitable for
     the Derivative Spectra function, and running the DS function."""

  # Parse out the water pixels in Sentinel 2 A/B MSI image.

  # This function masks the cloudy pixels and identifies the water pixels.
  def get_water(image):

    # Water with cloud check, but note this is the not bit addition method.
    water_pixels = (image.select('QA60').eq(0).And(image.select('SCL').eq(6)))

    # water_pixels=image.select('QA60')
    # cloudBitMask=1 << 10
    # cirrusBitMask=1 << 11
    # mask = water_pixels.bitwiseAnd(cloudBitMask).eq(0).And(water_pixels.bitwiseAnd(cirrusBitMask).eq(0))

    # Mask all non-water pixels in the image.
    image = image.updateMask(water_pixels)

    return image.select(non_ee_band_list).toDouble()

  # Run the LS8 mask.
  water_mask = get_water(water)

  # ======================================================== #

  # Using this for the subtraction of average values, addition of weighted average values.

  # Making an image out of the average values. This creates a constant image.

  band_averages = water_mask.reduceRegion(ee.Reducer.mean(),roi,maxPixels = maxP, scale = 10).values().getInfo()

  single_image_averages = ee.ImageCollection([ee.Image(value) for value in band_averages]).toBands()

  # ======================================================== #

  # Used this for exporting the band values used for the weighted averaging.

  forWeighting = ee.Feature(None,
                            {'band_values': water_mask.reduceRegion(ee.Reducer.mean(),roi,maxPixels = maxP, scale = 10).values(),
                             'pixel_total': water_mask.reduceRegion(ee.Reducer.count(),roi,maxPixels = maxP, scale = 10).get('B1') })

  # ***** #
  # Names for this export section...
  runtime_filename = f'6VPCA_{siteName_2char}_{water.get("name").getInfo()}_bandAverages_{bandCombo}'

  # ***** #
  avgs = batch.Export.table.toDrive(collection = ee.FeatureCollection(forWeighting),
                              fileFormat = 'CSV',
                              description = runtime_filename)

  batch.Task.start(avgs)

  # Move the file to its new home once the file is finished computing in Earth Engine.
  i = 1
  while avgs.status()['state'] in ['READY', 'RUNNING']:
    sys.stdout.write("\r" + f'Band average export status update #{i}: ' + str(avgs.status()))
    sys.stdout.flush()
    i += 1
    time.sleep(5)

  else:

    print('Band average export completed...')
    print(avgs.status())

  # Append runtime filename
  return runtime_filename

  # ======================================================== #

In [ ]:
# List our directory of folders, which we can utilize to write files.
im_folders = os.listdir(f'{output_folder}')
img_names = [img.get("name").getInfo() for img in im_list]
im_folders = [name+'_outputs' for name in img_names]
# for name in img_names:
#   os.mkdir(f'{output_folder}/{name}_outputs')

In [ ]:
# Put filenames in here for moving.
forMoving = []

# Use this for exporting average band values.
for i in range(len(im_folders)):
  forMoving += [ derivative_images(ee.Image(im_list[i])) ]

Band average export status update #9: {'state': 'RUNNING', 'description': '6VPCA_IRL_20200101T160511_20200101T160506_T17RNM_bandAverages_B2-B8A', 'creation_timestamp_ms': 1680106775796, 'update_timestamp_ms': 1680106818565, 'start_timestamp_ms': 1680106818533, 'task_type': 'EXPORT_FEATURES', 'attempt': 1, 'id': 'U7JUROHIK722GQYPBZXMACCF', 'name': 'projects/earthengine-legacy/operations/U7JUROHIK722GQYPBZXMACCF'}Band average export completed...
{'state': 'COMPLETED', 'description': '6VPCA_IRL_20200101T160511_20200101T160506_T17RNM_bandAverages_B2-B8A', 'creation_timestamp_ms': 1680106775796, 'update_timestamp_ms': 1680106822884, 'start_timestamp_ms': 1680106818533, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/'], 'attempt': 1, 'batch_eecu_usage_seconds': 87.19464111328125, 'id': 'U7JUROHIK722GQYPBZXMACCF', 'name': 'projects/earthengine-legacy/operations/U7JUROHIK722GQYPBZXMACCF'}
Band average export status update #4: {'state': 'RUNNING', 'description': 

In [ ]:
for i in range(len(forMoving)):
  os.rename(f'/content/gdrive/My Drive/{forMoving[i]}.csv',f'{output_folder}/{im_folders[i]}/{forMoving[i]}.csv')

After this, run [FloridaDEP_calculatingWeightedAverages.ipynb](https://colab.research.google.com/drive/1BBQ2_9basbC9dqnyl3wJgTH3eLg9_U7H)